In [648]:
import pandas as pd
import random
from datetime import datetime, timedelta

In [649]:
# CRIA FUNÇÕES DE ACESSO AO BANCO DE DADOS
import psycopg2

# cria conexão
def conn():
  con = psycopg2.connect(
    database = "drogasystem", 
    user = "oscar", 
    password = "oscar", 
    host = "127.0.0.1", 
    port = "5432")
  cur = con.cursor()
  return con, cur

# faz selects
def select(sql):
  all = []
  con, cur = conn()
  cur.execute(sql)
  all = cur.fetchall()
  cur.close()
  con.close()
  return all

# executa inserts e updates
def exec(sql):
  try:
    con, cur = conn()
    cur.execute(sql)
    con.commit()
  except (Exception, psycopg2.Error) as error:
    if cur == None: cur.rollback()
    print("Error:", error)
  finally:
    if cur: cur.close()
    if con: con.close()

In [650]:
# CRIA DATAFRAME DA TABELA DE MEDICAMENTO
cols = ["ID", "LABORATORIO_ID", "NOME", "SUBSTANCIA", "DOSAGEM", "APRESENTACAO", 
        "TIPO", "CONTROLE", "QUANTIDADE", "GGREM", "CLASSE", "PRECO"]
sql = f"select {','.join(cols)} from medicamento order by 1;"
df_medicamento = pd.DataFrame(select(sql), columns=cols)
df_medicamento.head()

,ID,LABORATORIO_ID,NOME,SUBSTANCIA,DOSAGEM,APRESENTACAO,TIPO,CONTROLE,QUANTIDADE,GGREM,CLASSE,PRECO
0,1,1,AMPICILINA,AMPICILINA,50 MG/ML,Líquido,Genérico,AM,150 ML,507701907133117,PENICILINAS ORAIS DE AMPLO ESPECTRO,74.11
1,2,2,NESINA MET,BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA,"(12,5 + 1000,0) MG",Sólido,Novo,Liberado,60,501115110025705,ASSOCIAÇÕES DE INIBIDORES DPP-IV COM BIGUANIDAS,303.77
2,3,3,TEGRETOL,CARBAMAZEPINA,200 MG,Sólido,Novo,C1,60,526515003110311,ANTIEPILÉPTICOS,89.22
3,4,4,TRELIM,CLORIDRATO DE SERTRALINA,100 MG,Sólido,Similar,C1,60,528519120163307,ANTIDEPRESSIVOS SSRI,401.24
4,5,5,ZETERINA,EZETIMIBA,10 MG,Sólido,Similar,Liberado,60,543817070024603,TODOS OUTROS REGULADORES DE COLESTEROL E TRIGL...,277.35


In [651]:
# CRIA DATAFRAME DO EXCEL
df_excel = pd.read_excel("cmed_final_v3.xlsx"
          ).filter(items=[4, 11, "Probabilidade", "Validade"]
          ).rename(columns={4: "GGREM", 11: "CUSTO", "Probabilidade": "PROBABILIDADE", "Validade": "VALIDADE"})
df_excel.head()

,GGREM,CUSTO,PROBABILIDADE,VALIDADE
0,507701907133117,53.61,2.5,180
1,501115110025705,228.00,2.5,180
2,526515003110311,64.54,2.5,180
3,528519120163307,290.24,2.5,180
4,543817070024603,200.62,2.5,180


In [652]:
# CRIA FUNÇÃO INSERE ESTOQUE
def insert_estoque(i, data):
  # acessa medicamento_id
  medicamento_id = int(df_medicamento.loc[i, "ID"])

  # cria lote
  lote = f"lote {random.randrange(0, 999999):06d}"

  # procura linha do excel com base no ggrem
  excel = df_excel[df_excel["GGREM"] == df_medicamento.loc[i, "GGREM"]].reset_index(drop=True)

  # acessa custo
  custo = float(excel["CUSTO"][0])

  # calcula data da validade
  validade = int(excel["VALIDADE"][0])
  validade = data + timedelta(days = validade)
  
  # calcula quant_inicial
  probabilidade = float(excel["PROBABILIDADE"][0])
  quant_inicial = random.randrange(10, int(probabilidade * 10), 10)

  # gera insert do estoque
  sql = "insert into estoque values "
  sql += f"(default, {medicamento_id}, '{lote}', '{data}', '{validade}', {custo}, {quant_inicial}, 0, {quant_inicial}); \n"

  # gera insert ou update do estoque_total
  if select(f"select count(*) from estoque_total where medicamento_id = {medicamento_id};")[0][0] == 0:
    minimo = 5 if custo > 100 else 10
    sql += f"insert into estoque_total values (default, {medicamento_id}, {quant_inicial}, {minimo}, 9999); "
  else:
    sql += f"update estoque_total set total = total + {quant_inicial} where medicamento_id = {medicamento_id}; "
    
  return sql + "\n"

In [653]:
# GERA INSERTS DO ESTOQUE INICIAL
if select("select count(*) from estoque;")[0][0] == 0:
  sql = ""
  data = datetime.strptime("03/01/2023", "%d/%m/%Y").date()
  for i in df_medicamento.index: sql += insert_estoque(i, data)
  print(sql)
  exec(sql)

In [654]:
# CRIA DATAFRAME DA TABELA DE CLIENTE
df_cliente = pd.DataFrame(select("select id from cliente;"), columns=["ID"])
df_cliente.head()

,ID
0,1
1,2
2,3
3,4
4,5


In [655]:
# CRIA DATAFRAME DA TABELA DE PROFISSIONAL
df_profissional = pd.DataFrame(select("select id from profissional;"), columns=["ID"])
df_profissional.head()

,ID
0,21
1,22
2,23
3,24
4,25


In [656]:
# CRIA DATAFRAME DA TABELA DE ESTOQUE
cols = ["ESTOQUE_ID", "MEDICAMENTO_ID", "QUANT_ATUAL", "GGREM", 
        "PRECO", "CONTROLE", "MINIMO", "PROBABILIDADE"]

# produtos em estoque
sql  = "select estoque.id, estoque.medicamento_id, estoque.quant_atual, "
sql += "medicamento.ggrem, medicamento.preco, medicamento.controle, "
sql += "estoque_total.minimo, 0.0 "
sql += "from estoque "
sql += "inner join medicamento on medicamento.id = estoque.medicamento_id "
sql += "inner join estoque_total on estoque_total.medicamento_id = estoque.medicamento_id "
sql += "where quant_atual > 0 "
sql += "and estoque.id in (select min(id) from estoque "
sql += "where quant_atual > 0 group by medicamento_id) "
sql += "order by 1;"

df_estoque = pd.DataFrame(select(sql), columns=cols)

# atualiza df com valor da probabilidade
for i in df_estoque.index:
  excel = df_excel[df_excel["GGREM"] == df_estoque.loc[i, "GGREM"]].reset_index(drop=True)
  df_estoque.loc[i, "PROBABILIDADE"] = excel["PROBABILIDADE"][0]

df_estoque.head()

,ESTOQUE_ID,MEDICAMENTO_ID,QUANT_ATUAL,GGREM,PRECO,CONTROLE,MINIMO,PROBABILIDADE
0,1,1,10,507701907133117,74.11,AM,10,2.5
1,2,2,20,501115110025705,303.77,Liberado,5,2.5
2,3,3,20,526515003110311,89.22,C1,10,2.5
3,4,4,10,528519120163307,401.24,C1,5,2.5
4,5,5,10,543817070024603,277.35,Liberado,5,2.5


In [678]:
# CRIA FUNÇÃO INSERE VENDA
def insert_venda(data_hora, venda_id):
  sql = ""
  totais = 0

  # cliente e profissional
  cliente_id = "null"
  profissional_id = "null"
  if random.randrange(10) > 5:
    cliente_id = df_cliente.loc[random.choice(df_cliente.index), 'ID']

  # quantidade de itens da venda
  quantidade_itens_venda = random.randrange(3) + 1
  for q in range(quantidade_itens_venda):
    # escolha do item
    i = random.choices(df_estoque.index, weights=df_estoque["PROBABILIDADE"])[0]
    
    estoque_id = int(df_estoque.loc[i, "ESTOQUE_ID"])
    medicamento_id = int(df_estoque.loc[i, "MEDICAMENTO_ID"])
    quant_atual = int(df_estoque.loc[i, "QUANT_ATUAL"])
    preco = float(df_estoque.loc[i, "PRECO"])
    controle = str(df_estoque.loc[i, "CONTROLE"])
    minimo = int(df_estoque.loc[i, "MINIMO"])

    if controle != "Liberado": 
      profissional_id = df_profissional.loc[random.choice(df_profissional.index), 'ID']

    # calcula quantidade
    if preco > 100:
      quantidade = 1
    else:
      quantidade = random.randrange(3) + 1

    desconto = 0
    if cliente_id != "null":
      desconto = quantidade * preco * random.randrange(40) / 100
    elif random.randrange(10) > 0:
      desconto = quantidade * preco * random.randrange(10) / 100
    desconto = int(desconto * 10) / 10

    if quantidade > quant_atual:
      quantidade = quant_atual
    
    # totais
    total = quantidade * preco - desconto
    totais += total

    # cria insert e update do item de venda
    sql += "insert into itens_venda values "
    sql += f"(default, {venda_id}, {medicamento_id}, {estoque_id}, {quantidade}, {desconto:.2f}, {total:.2f}); \n"
    sql += f"update estoque set quant_venda = quant_venda + {quantidade}, quant_atual = quant_atual - {quantidade} where id = {estoque_id}; \n"
    sql += f"update estoque_total set total = total - {quantidade} where medicamento_id = {medicamento_id}; \n"

    if quant_atual - quantidade <= minimo:
      # procura índice do medicamento
      j = df_medicamento[df_medicamento["GGREM"] == df_estoque.loc[i, "GGREM"]].index[0]
      sql += insert_estoque(j, data_hora.date())

  sql = f"insert into venda values ({venda_id}, '{data_hora}', {totais:.2f}, {cliente_id}, {profissional_id}); \n" + sql
  return sql

In [686]:
# data inicial das vendas
data = select("select coalesce(max(time_stamp), '2022-01-02') from venda;")[0][0]

# venda_id inicial
venda_id = select("select coalesce(max(id), 0) as id from venda;")[0][0]

sql = ""

# período de venda
for i in range(200):
  data += timedelta(days = 1)

  # começa as vendas pela manhã
  data_hora = data + timedelta(hours = 6)

  # quantidade de vendas no dia
  qtd_vnd_dia = random.randrange(10)
  for j in range(qtd_vnd_dia):
    venda_id += 1

    # intervalo entre cada venda
    data_hora += timedelta(seconds = random.randrange(10000))

    sql += insert_venda(data_hora, venda_id)

print(sql)
exec(sql)

insert into venda values (2001, '2023-03-21 17:07:29', 47.88, null, null); 
insert into itens_venda values (default, 2001, 469, 469, 1, 0.00, 47.88); 
update estoque set quant_venda = quant_venda + 1, quant_atual = quant_atual - 1 where id = 469; 
update estoque_total set total = total - 1 where medicamento_id = 469; 
insert into venda values (2002, '2023-03-21 19:33:44', 168.61, 20, 21); 
insert into itens_venda values (default, 2002, 274, 274, 1, 12.60, 23.50); 
update estoque set quant_venda = quant_venda + 1, quant_atual = quant_atual - 1 where id = 274; 
update estoque_total set total = total - 1 where medicamento_id = 274; 
insert into itens_venda values (default, 2002, 213, 213, 1, 25.50, 145.11); 
update estoque set quant_venda = quant_venda + 1, quant_atual = quant_atual - 1 where id = 213; 
update estoque_total set total = total - 1 where medicamento_id = 213; 
insert into venda values (2003, '2023-03-21 22:03:10', 565.20, null, null); 
insert into itens_venda values (default

In [676]:
df_profissional.loc[random.choice(df_profissional.index), 'ID']

24

In [8]:
df = pd.DataFrame(columns=['a', 'b'])
for i in range(1000):
  df.loc[df.size] = [1 * i, 2 * i]
df

,a,b
0,0,0
2,1,2
4,2,4
6,3,6
8,4,8
...,...,...
1990,995,1990
1992,996,1992
1994,997,1994
1996,998,1996


In [322]:
x = [111,222,333,444,555,666]
y = {111: 0, 222: 0, 333: 0, 444: 0, 555: 0, 666: 0}
for i in range(1000):
  y[random.choices(x, weights=(1, 2, 3, 4, 5, 6))[0]] += 1
print(y)

{111: 54, 222: 88, 333: 131, 444: 209, 555: 221, 666: 297}


In [10]:
df = pd.read_excel("cmed_final_v3.xlsx")
df = df.filter(items=[4, 'Probabilidade'])
df = df.rename(columns={4: 'GGREM', 'Probabilidade': 'PROBABILIDADE'})
df.head()

,GGREM,PROBABILIDADE
0,507701907133117,2.5
1,501115110025705,2.5
2,526515003110311,2.5
3,528519120163307,2.5
4,543817070024603,2.5


In [3]:
# atualizar estoque
def estoque(dat, med_id, preco, fval, fpre):
  # lote
  lot = random.randrange(100000, 999999)

  # data da validade
  val = dat + timedelta(days = fval * 30 + 30)
  val = datetime.strptime(val.strftime('1/%m/%Y'), '%d/%m/%Y').date() - timedelta(days = 1)
  
  # preco de compra
  pre = float(preco)
  pre = int(pre / fpre * 10)
  pre /= 10

  # quantidade comprada
  if preco > 600:
    qtd = random.randrange(2, 7, 2)
  elif preco > 300:
    qtd = random.randrange(3, 10, 3)
  elif preco > 100:
    qtd = random.randrange(5, 16, 5)
  elif preco > 40:
    qtd = random.randrange(2, 20, 4)
  else:
    qtd = random.randrange(3, 49, 4)

  sql = "insert into estoque (medicamento_id, lote, data, validade, preco, quant_inicial, quant_atual) "
  sql += f"values ({med_id}, 'lote {lot}', '{dat}', '{val}', {pre}, {qtd}, {qtd}); \n"

  # if model.find_by_sql(f"select count(*) from estoque_total where medicamento_id = {med_id}; ")[0][0] == 0:
  if select(f"select count(*) from estoque_total where medicamento_id = {med_id};")[0][0] == 0:
    if preco > 400:
      min = 3
    elif preco > 100:
      min = 5
    else:
      min = 10
    sql += f"insert into estoque_total (medicamento_id, total, minimo) values ({med_id}, {qtd}, {min}); "
  else:
    sql += f"update estoque_total set total = total + {qtd} where medicamento_id = {med_id}; "
    
  return sql

In [305]:
# venda
def venda(clis, pros, dat, venda_id):
  sql = ""
  sql1 = ""
  sql2 = ""
  sql3 = ""
  total = 0

  # cliente e profissional
  pro = None
  cli = None
  if random.randrange(3) > 0:
    cli = str(clis[random.randrange(len(clis))][0])

  # produtos em estoque
  s = "select estoque.id, estoque.medicamento_id, estoque.quant_atual, medicamento.preco, medicamento.controle, "
  s += "estoque.preco, estoque.data, estoque.validade, estoque_total.minimo from estoque "
  s += "inner join medicamento on medicamento.id = estoque.medicamento_id "
  s += "inner join estoque_total on estoque_total.medicamento_id = estoque.medicamento_id "
  s += "where quant_atual > 0 order by 1;"
  ests = select(s)

  # quantidade de itens da venda
  qiv = random.randrange(3) + 1
  for i in range(qiv):

    # escolha do produto
    p = random.randrange(len(ests))

    est_id = ests[p][0]
    med_id = ests[p][1]
    qtd_atu = ests[p][2]
    preco = float(ests[p][3])
    contr = ests[p][4]
    
    # VERIFICAR SE É MEDICAMENTO CONTROLADO!!!
    # profissional
    if contr != "Não controlado":
      pro = str(pros[random.randrange(len(pros))][0])

    # quantidade do produto
    if preco > 200:
      qtd_vnd = random.randrange(3) + 1
    elif preco > 50:
      qtd_vnd = random.randrange(5) + 1
    else:
      qtd_vnd = random.randrange(10) + 1
    if qtd_vnd > qtd_atu:
      qtd_vnd = qtd_atu

    # desconto para cliente de até 40% e 10% de chance dar desconto para não cliente até 10%
    desc = 0
    if cli:
      desc = qtd_vnd * preco * random.randrange(40) / 100
      desc = int(desc * 10) / 10
    elif random.randrange(10) > 0:
      desc = qtd_vnd * preco * random.randrange(10) / 100
      desc = int(desc * 10) / 10

    # totais
    tot = qtd_vnd * preco - desc
    total += tot

    sql1 += "insert into itens_venda (venda_id, medicamento_id, estoque_id, quantidade, desconto, total) "
    sql1 += f"values ({venda_id}, {med_id}, {est_id}, {qtd_vnd}, {desc:.2f}, {tot:.2f}); \n"
    sql1 += f"update estoque set quant_venda = quant_venda + {qtd_vnd}, quant_atual = quant_atual - {qtd_vnd} where id = {est_id}; \n"
    sql1 += f"update estoque_total set total = total - {qtd_vnd} where medicamento_id = {med_id}; \n"

    # reposição do estoque
    if qtd_atu < ests[p][8]:
      pr = ests[p][3]
      fval = (ests[p][7] - ests[p][6]).days / 30
      fpre = float(ests[p][3] / ests[p][5])
      sql2 += estoque(dat.date(), med_id, pr, fval, fpre) + "\n"
  
  sql3 = f"insert into venda ({'cliente_id, ' if cli else ''}{'profissional_id, ' if pro else ''}time_stamp, total) "
  sql3 += f"values ({cli + ', ' if cli else ''}{pro + ', ' if pro else ''}'{dat}', {total:.2f}); \n"
  sql = sql3 + sql1 + sql2
  print(sql)
  exec(sql)

In [5]:
if select(f"select count(*) from estoque;")[0][0] == 0:
  print("\n-- estoque inicial")
  meds = select(f"select id, nome, preco from medicamento order by 1;")
  
  dic = {}
  for med in meds:
    if (not med[1] in dic):
      dic[med[1]] = {}
      dic[med[1]]["val"] = random.randrange(6, 37, 6) # fator de validade
      dic[med[1]]["pre"] = random.uniform(1.8, 2.9) # fator de preço
      
  for med in meds:
    # data da compra
    dat = datetime.strptime('31/07/23', '%d/%m/%y').date()
    sql = estoque(dat, med[0], med[2], dic[med[1]]["val"], dic[med[1]]["pre"])
    print(sql)
    exec(sql)


-- estoque inicial
insert into estoque (medicamento_id, lote, data, validade, preco, quant_inicial, quant_atual) values (1, 'lote 966265', '2023-07-31', '2024-07-31', 26.9, 6, 6); 
insert into estoque_total (medicamento_id, total, minimo) values (1, 6, 10); 
insert into estoque (medicamento_id, lote, data, validade, preco, quant_inicial, quant_atual) values (2, 'lote 739498', '2023-07-31', '2026-07-31', 146.9, 3, 3); 
insert into estoque_total (medicamento_id, total, minimo) values (2, 3, 5); 
insert into estoque (medicamento_id, lote, data, validade, preco, quant_inicial, quant_atual) values (3, 'lote 994181', '2023-07-31', '2025-07-31', 30.9, 6, 6); 
insert into estoque_total (medicamento_id, total, minimo) values (3, 6, 10); 
insert into estoque (medicamento_id, lote, data, validade, preco, quant_inicial, quant_atual) values (4, 'lote 305287', '2023-07-31', '2025-01-31', 145.0, 9, 9); 
insert into estoque_total (medicamento_id, total, minimo) values (4, 9, 3); 
insert into estoque 

In [6]:
# Vendas
print("\n-- vendas, itens de venda")

# possiveis clientes e profissionais
clis = select("select id from cliente;")
pros = select("select id from profissional;")

# data inicial
if select("select count(*) from venda;")[0][0] == 0:
  dat = datetime.strptime('01/08/23', '%d/%m/%y')
else:
  dat = select("select max(time_stamp) from venda;")[0][0]
  dat = datetime.strptime(dat.strftime('%d/%m/%y'), '%d/%m/%y')

# venda_id inicial
venda_id = select("select coalesce(max(id), 0) as id from venda;")[0][0]

# período de venda
for i in range(90):
  dat += timedelta(days = 1)
  dat += timedelta(seconds = random.randrange(36000))

  # quantidade de vendas no dia
  qtd_vnd_dia = random.randrange(5)
  for j in range(qtd_vnd_dia):
    venda_id += 1
    dat += timedelta(seconds = random.randrange(14000))
    venda(clis, pros, dat, venda_id)


-- vendas, itens de venda
insert into venda (cliente_id, profissional_id, time_stamp, total) values (7, 21, '2023-08-02 11:33:33', 1066.49); 
insert into itens_venda (venda_id, medicamento_id, estoque_id, quantidade, desconto, total) values (1, 209, 209, 2, 33.10, 58.94); 
update estoque set quant_venda = quant_venda + 2, quant_atual = quant_atual - 2 where id = 209; 
update estoque_total set total = total - 2 where medicamento_id = 209; 
insert into itens_venda (venda_id, medicamento_id, estoque_id, quantidade, desconto, total) values (1, 47, 47, 3, 66.80, 675.61); 
update estoque set quant_venda = quant_venda + 3, quant_atual = quant_atual - 3 where id = 47; 
update estoque_total set total = total - 3 where medicamento_id = 47; 
insert into itens_venda (venda_id, medicamento_id, estoque_id, quantidade, desconto, total) values (1, 46, 46, 2, 82.90, 331.94); 
update estoque set quant_venda = quant_venda + 2, quant_atual = quant_atual - 2 where id = 46; 
update estoque_total set total 